Aim: Predict if PM concentration is higher than boundary value based on meteorologic parameters, leave out influence of traffic data.
daily boundary value for PM10:  50 µg/m³

In [156]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#import seaborn as sn
import matplotlib.pyplot as plt

%matplotlib inline

In [227]:
os.chdir(r"C:\Users\Anna\Documents\UNI\MA Semi 1\Machine Learning\Final_project")

## test file from luftdateninfo
file_pm = "test_PM_2020-01-23_sds011_sensor_10963.csv"
pm_ds = pd.read_csv(file_pm, sep=";", index_col="timestamp")

## df with time_col as index
pm_ds = pm_ds[["P1"]]

#Sum up to hourly values
pm_ds.index = pd.to_datetime(pm_ds.index) # convert index into time_dtype
print(pm_ds)

###### needs to be chnaged- save timestamp  #########
pm_ds_hour = pm_ds.groupby(pm_ds.index.hour).mean()# ds= one day -->24 rows
print(pm_ds_hour)


#pm_ds_hour.index = pd.to_datetime(pm_ds_hour.index, format="%Y-%m-%d %H:%M:%S")#, unit="h")
pm_ds_hour.index = pd.to_datetime(pm_ds_hour.index,unit="h")
print(pm_ds_hour)


                        P1
timestamp                 
2020-01-23 00:02:21  57.70
2020-01-23 00:04:57  58.93
2020-01-23 00:07:26  60.67
2020-01-23 00:10:05  58.90
2020-01-23 00:13:07  57.97
...                    ...
2020-01-23 23:47:57  31.97
2020-01-23 23:50:30  38.50
2020-01-23 23:53:15  36.90
2020-01-23 23:56:03  40.73
2020-01-23 23:58:32  36.67

[555 rows x 1 columns]
                  P1
timestamp           
0          61.175455
1          60.337778
2          60.901000
3          58.587727
4          53.806000
5          42.638333
6          42.976667
7          45.372917
8          43.117500
9          46.309130
10         38.529583
11         33.917500
12         31.267083
13         28.494000
14         26.772917
15         25.041667
16         23.417917
17         32.340417
18         42.986818
19         40.723913
20         43.877083
21         47.411250
22         48.723333
23         38.984737


Load meteorologic test-file and merge both ds (PM + meteo)

In [211]:
file_wind =  pd.read_csv("./DWDstats_meteo/produkt_ff_stunde_19530101_20191231_04931.txt", sep=";", header=0,
                         index_col="MESS_DATUM")
#print(file_wind)
file_rain = pd.read_csv("./DWDstats_meteo/produkt_rr_stunde_19951004_20191231_04931.txt", sep=";", header=0,
                         index_col="MESS_DATUM")
file_pressure = pd.read_csv("./DWDstats_meteo/produkt_p0_stunde_19490101_20191231_04931.txt", sep=";", header=0,
                          index_col="MESS_DATUM") 
file_temperature = pd.read_csv("./DWDstats_meteo/produkt_tu_stunde_19880101_20191231_04931.txt", sep=";", header=0,
                          index_col="MESS_DATUM") 

allmeteo = file_wind.merge(file_rain, on ="MESS_DATUM").merge(file_pressure, on ="MESS_DATUM").merge(file_temperature, on ="MESS_DATUM")
allmeteo = allmeteo[["   F", "   D", "  R1", "  P0", "TT_TU", "RF_TU"]]
allmeteo = allmeteo.rename(columns={"   F": "F", "   D":"D", "  R1": "R1", "  P0": "P0"})

print(allmeteo)


              F    D   R1     P0  TT_TU  RF_TU
MESS_DATUM                                    
1995100414  2.6  220  0.0  970.6   23.3   51.0
1995100415  3.6  210  0.0  970.3   23.8   50.0
1995100416  1.5  150  0.0  969.9   23.0   54.0
1995100417  1.5  120  0.0  970.0   21.5   63.0
1995100418  1.5  180  0.0  970.1   19.4   71.0
...         ...  ...  ...    ...    ...    ...
2019123119  1.3  270  0.0  987.3    0.3   82.0
2019123120  0.9  270  0.0  987.8    0.9   84.0
2019123121  1.0  230  0.0  987.2    1.2   87.0
2019123122  0.9  300  0.0  987.5    1.4   87.0
2019123123  0.5  230  0.0  987.3    1.1   89.0

[211931 rows x 6 columns]


In [217]:

## convert to same datetime
allmeteo.index = pd.to_datetime(allmeteo.index, format="%Y%m%d%H") # errors='ignore')
print(type(pm_ds_hour.index))

allmeteo2_hour = allmeteo2 #allmeteo2.groupby(allmeteo2.index.hour).mean()# ds= one day -->24 rows
allmeteo2_hour.index = pd.to_datetime(allmeteo2_hour.index, unit="h")
print(allmeteo2_hour.index)


# ### merge both DS (meteo Ds + PM DS)
# print(type(allmeteo.index))
PM_meteo = pm_ds_hour.merge(allmeteo, left_index = True, right_index=True) 
print(PM_meteo)

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['1995-10-04 14:00:00', '1995-10-04 15:00:00',
               '1995-10-04 16:00:00', '1995-10-04 17:00:00',
               '1995-10-04 18:00:00', '1995-10-04 19:00:00',
               '1995-10-04 20:00:00', '1995-10-04 21:00:00',
               '1995-10-04 22:00:00', '1995-10-04 23:00:00',
               ...
               '2019-12-31 14:00:00', '2019-12-31 15:00:00',
               '2019-12-31 16:00:00', '2019-12-31 17:00:00',
               '2019-12-31 18:00:00', '2019-12-31 19:00:00',
               '2019-12-31 20:00:00', '2019-12-31 21:00:00',
               '2019-12-31 22:00:00', '2019-12-31 23:00:00'],
              dtype='datetime64[ns]', name='MESS_DATUM', length=211931, freq=None)
Empty DataFrame
Columns: [P1, F, D, R1, P0, TT_TU, RF_TU]
Index: []


In [ ]:
#####################################

In [232]:
## vorläufig
PM_meteo = allmeteo
PM_meteo["P1"] = np.random.randint(0, 5, PM_meteo.shape[0])
PM_meteo

,F,D,R1,P0,TT_TU,RF_TU,P1
MESS_DATUM,,,,,,,
1995-10-04 14:00:00,2.6,220,0.0,970.6,23.3,51.0,3
1995-10-04 15:00:00,3.6,210,0.0,970.3,23.8,50.0,4
1995-10-04 16:00:00,1.5,150,0.0,969.9,23.0,54.0,2
1995-10-04 17:00:00,1.5,120,0.0,970.0,21.5,63.0,4
1995-10-04 18:00:00,1.5,180,0.0,970.1,19.4,71.0,4
...,...,...,...,...,...,...,...
2019-12-31 19:00:00,1.3,270,0.0,987.3,0.3,82.0,1
2019-12-31 20:00:00,0.9,270,0.0,987.8,0.9,84.0,4
2019-12-31 21:00:00,1.0,230,0.0,987.2,1.2,87.0,2


In [257]:
# split the boston hoursing dataset into training and testing 
x = PM_meteo[["F", "D","R1", "P0", "TT_TU", "RF_TU"]]#.values
y = PM_meteo["P1"]#.values


ratio = 0.2 # split ratio
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=ratio,random_state=5)

#print the size of train and test dataset

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(169544, 6)
(169544,)
(42387, 6)
(42387,)


In [258]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xstan = scaler.fit_transform(x)

In [262]:
dataStan=pd.DataFrame(data = Xstan, columns = PM_meteo.columns[0:6]) #["F", "D","R1", "P0", "TT_TU", "RF_TU"]])
dataStan["P1"]=y
dataStan.head()

softReg = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')
softReg.fit(Xstan,y)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [263]:
softReg.intercept_, softReg.coef_


(array([-0.00256376,  0.00679844, -0.00910539, -0.00051189,  0.0053826 ]),
 array([[-0.00473569,  0.00073119, -0.00072512,  0.00176374,  0.00186578,
         -0.00044391],
        [-0.00061064,  0.00389722, -0.00165798,  0.00476586,  0.00238907,
         -0.00118316],
        [ 0.00334544,  0.00416248,  0.00714612, -0.00476839, -0.00332466,
          0.00537091],
        [-0.00206968, -0.00843612, -0.00022594,  0.00115601, -0.00239714,
          0.00050719],
        [ 0.00407058, -0.00035477, -0.00453707, -0.00291723,  0.00146695,
         -0.00425103]]))

In [268]:
yhat = softReg.predict(Xstan)
dataStan['predict'] = yhat
dataStan.head()


,F,D,R1,P0,TT_TU,RF_TU,P1,predict
0,0.025015,0.284275,0.010297,0.002529,1.622711,-1.048298,NaN,1
1,0.067571,0.185870,0.010297,-0.019185,1.683568,-1.091187,NaN,4
2,-0.021796,-0.404558,0.010297,-0.048137,1.586197,-0.919631,NaN,4
3,-0.021796,-0.699773,0.010297,-0.040899,1.403628,-0.533632,NaN,4
4,-0.021796,-0.109344,0.010297,-0.033661,1.148030,-0.190521,NaN,1


Evaluation of Softmax permorfance

In [269]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(dataStan["P1"].values, yhat)
print(cm)

# confusionMatrix = pd.DataFrame(data = cm, index=['poor(0), true','good(1), true','great(2), true'], columns = ['poor(0), predicted','good(1), predicted','great(2), predicted'])
# confusionMatrix.loc['sum'] = confusionMatrix.sum()
# confusionMatrix['sum'] = confusionMatrix.sum(axis=1)
# confusionMatrix


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').